In [20]:
##### Written by Aris Dimou

import os, numpy as np, re, math, sys, glob, subprocess as sbp, copy

In [22]:
def Create_VASP(Structure, rep, i_dw, file_n, Relax_at = ['T', 'T', 'T']):
    
    ### Lambda functions 
    
    l_coord = (lambda x: f'{x:5f}') ## turns coordinates into strings   
    
    ### Atoms & Simulation box
    
    A_line, B_line, O_line = [], [], []

    rep_x, rep_y, rep_z, uc = rep[0], rep[1], rep[2], 5
    
    NofA = uc*rep_x*rep_y*rep_z               # Number of atoms
    N_cells = rep_x*rep_y*rep_z 

    if Structure == 'Cubic': ### LDA
        
        a, b, c = 3.96769, 3.96769, 3.96769
        xy, xz, yz = 0.000000000000, 0.000000000000, 0.000000000000
        
        x  = np.array((0.0, 0.5, 0.0, 0.5, 0.5)) #all x Ba, Ti, O1, O2, O3
        x_ = np.array((0.0, 0.5, 0.0, 0.5, 0.5)) 
        
        y  = np.array((0.0, 0.5, 0.5, 0.0, 0.5)) 
        y_ = np.array((0.0, 0.5, 0.5, 0.0, 0.5))
        
        z  = np.array((0.0, 0.5, 0.5, 0.5, 0.0)) 
        z_ = np.array((0.0, 0.5, 0.5, 0.5, 0.0))

    elif Structure == 'T_180': ### LDA
        
        a, b, c = 3.94568, 3.94568, 3.98252 
        xy, xz, yz = 0.000000000000, 0.000000000000, 0.000000000000
        
        x  = np.array([0.0, 0.5, 0.0, 0.5, 0.5])
        x_ = np.array([0.0, 0.5, 0.0, 0.5, 0.5])
        
        y  = np.array((0.0, 0.5, 0.5, 0.0, 0.5))
        y_ = np.array((0.0, 0.5, 0.5, 0.0, 0.5))
        
        z = np.array([-0.00239, 0.50851, 0.48643, 0.48643, -0.01982])
        z_ = np.array([0.00239, 0.49149, 0.51357, 0.51357, 0.01982])
        
    elif Structure == 'O_180': ### ACE
        
        a, b, c = 3.975301, 4.025221, 4.025221 
        xy, xz, yz = 0.000000000000, 0.000000000000, 0.0073281283782432
        
        x  = np.array([0.000000, 0.500000, 0.000000, 0.500000, 0.500000])
        x_ = np.array([0.000000, 0.500000, 0.000000, 0.500000, 0.500000])
        
        y  = np.array((-0.000010, 0.489020, 0.510980,  0.021339, 0.486137))
        y_ = np.array([ 0.000010, 0.514460, 0.489020, -0.021339, 0.513863])
        
        z  = np.array([-0.000010, 0.485540, 0.510980, 0.510980,  0.021339])
        z_ = np.array([ 0.000010, 0.514460, 0.489020, 0.489020, -0.021339])
        
       
    elif Structure == 'O_90': ### ACE
        
        a, b, c = 4.025221, 3.975301, 4.025221
        xy, xz, yz = 0.000000000000, 0.0073281283782432, 0.000000000000     
        
        x   = np.array([ 0.000010, 0.514460, 0.000000, -0.021339, 0.513863])
        x_  = np.array([ 0.000010, 0.514460, 0.000000, -0.021339, 0.513863])
        
        y  = np.array(( 0.000000, 0.500000, 0.500000,  0.000000, 0.500000))
        y_ = np.array([ 0.000000, 0.500000, 0.500000,  0.000000, 0.500000])
        
        z  = np.array([-0.000010, 0.485540, 0.510980, 0.510980,  0.021339])
        z_ = np.array([ 0.000010, 0.514460, 0.489020, 0.489020, -0.021339])           
        
    else:    
        
        print('Error Structure not defined')
    
    with open(file_n, 'w') as fw:
        
        fw.write('#Created by Aris Dimou\n')
        fw.write('1.00000\n' )
        fw.write('%f %f %f\n'%(round(rep_x*a, 5), round(xy,7), round(xz,7)))
        fw.write('%f %f %f\n'%(round(xy,7), round(rep_y*b, 5), round(yz,7)))
        fw.write('%f %f %f\n'%(round(xz,7), round(yz,7), round(rep_z*c, 5)))

        fw.write('Ba Ti O\n')
        fw.write('%i %i %i\n'%(N_cells, N_cells, N_cells*3))
        fw.write('Direct\n')
        
    old_id, c1, c2 = 1, 0, 0                                          

    for repz in range(0,rep_z):
        for repy in range(0, rep_y):
            for repx in range(0, rep_x):
                _i = 0

                for _id in range(old_id, old_id+uc): #str(format(x[0]+rep_x*a, '.7f'))
                    
                    t_line = []

                    if repx+1 <= i_dw:

                        t_xyz = [(x[_i]+repx)/(rep_x), (y[_i]+repy)/(rep_y),
                                 (z[_i]+repz)/(rep_z)]      

                        t_line = list(map(l_coord, t_xyz)) + Relax_at

                        c1 += 1


                    else:


                        t_xyz = [(x[_i]+repx)/(rep_x), (y[_i]+repy)/(rep_y),
                                 (z_[_i]+repz)/(rep_z)]      

                        t_line = list(map(l_coord, t_xyz)) + Relax_at

                        c2 += 1
                            
                    _i += 1

                    if _i == 1:
                        A_line.append(t_line)
                    elif _i == 2:
                        B_line.append(t_line)
                    elif _i > 2:
                        O_line.append(t_line)
                    
                
                old_id += uc
        
    with open(file_n, 'a') as fa:
        for l in A_line:
            line = ' '.join(l)+'\n'
            fa.write(line)
        for l in B_line:
            line = ' '.join(l) +'\n'
            fa.write(line)
        for l in O_line:
            line = ' '.join(l) +'\n'
            fa.write(line)   
            
    print(c1,c2)

In [23]:
####### Create_VASP(Structure, [x,y,z], where to place DW, filename, [])

os.chdir('/home/users/dimouadb/VASP/Test/Tetragonal/DSP/5x5x5/perfect/')
Create_VASP('T_180', [5,5,5], 0, 'POSCAR')


0 625
